# REINFORCE in TensorFlow

This notebook implements a basic reinforce algorithm a.k.a. policy gradient for CartPole env.

It has been deliberately written to be as simple and human-readable.


The notebook assumes that you have [openai gym](https://github.com/openai/gym) installed.

In case you're running on a server, [use xvfb](https://github.com/openai/gym#rendering-on-a-server)

In [1]:
import gym
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

env = gym.make("CartPole-v0")

#gym compatibility: unwrap TimeLimit
if hasattr(env,'env'):
    env=env.env

env.reset()
n_actions = env.action_space.n
state_dim = env.observation_space.shape

#plt.imshow(env.render("rgb_array"))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


# Building the policy network

For REINFORCE algorithm, we'll need a model that predicts action probabilities given states.

For numerical stability, please __do not include the softmax layer into your network architecture__. 

We'll use softmax or log-softmax where appropriate.

In [2]:
import tensorflow as tf

#create input variables. We only need <s,a,R> for REINFORCE
states = tf.placeholder('float32',(None,)+state_dim,name="states")
actions = tf.placeholder('int32',name="action_ids")
cumulative_rewards = tf.placeholder('float32', name="cumulative_returns")

In [3]:
#from keras.models import Sequential
#from keras.layers import Dense

#model = Sequential()
#model.add(Dense(4, input_dim=8, activation='relu'))

#<define network graph using raw tf or any deep learning library>

n_hidden_1 = 64

weights_1 = tf.Variable(tf.random_normal([state_dim[0], n_hidden_1]))
bias_1  =  tf.Variable(tf.random_normal([n_hidden_1]))

weights_2 = tf.Variable(tf.random_normal([n_hidden_1, n_actions]))
bias_2  =  tf.Variable(tf.random_normal([n_actions]))

activations = tf.nn.relu(tf.add(tf.matmul(states, weights_1), bias_1))
logits = tf.add(tf.matmul(activations, weights_2), bias_2)

policy = tf.nn.softmax(logits)
log_policy = tf.nn.log_softmax(logits)

In [4]:
#utility function to pick action in one given state
get_action_proba = lambda s: policy.eval({states:[s]})[0] 

#### Loss function and updates

We now need to define objective and update over policy gradient.

Our objective function is

$$ J \approx  { 1 \over N } \sum  _{s_i,a_i} \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$


Following the REINFORCE algorithm, we can define our objective as follows: 

$$ \hat J \approx { 1 \over N } \sum  _{s_i,a_i} log \pi_\theta (a_i | s_i) \cdot G(s_i,a_i) $$

When you compute gradient of that function over network weights $ \theta $, it will become exactly the policy gradient.


In [5]:
#get probabilities for parti
indices = tf.stack([tf.range(tf.shape(log_policy)[0]),actions],axis=-1)
log_policy_for_actions = tf.gather_nd(log_policy,indices)

In [6]:
# policy objective as in the last formula. please use mean, not sum.
# note: you need to use log_policy_for_actions to get log probabilities for actions taken

J = tf.reduce_mean(tf.multiply(log_policy_for_actions, cumulative_rewards))

In [7]:
#regularize with entropy
entropy = - tf.reduce_mean(tf.multiply(log_policy, policy))

In [8]:
#all network weights
all_weights = [weights_1, bias_1, weights_2, bias_2] #<a list of all trainable weights in your network>

#weight updates. maximizing J is same as minimizing -J. Adding negative entropy.
loss = -J - 0.1 * entropy

update = tf.train.AdamOptimizer().minimize(loss,var_list=all_weights)

### Computing cumulative rewards

In [9]:
def get_cumulative_rewards(rewards, #rewards at each step
                           gamma = 0.99 #discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently
    
    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    
    cumulative = [rewards[-1]]
    for r in reversed(rewards[:-1]):
        cumulative.append(r + gamma*cumulative[-1])
        
    return list(reversed(cumulative))
    
    

In [10]:
assert len(get_cumulative_rewards(range(100))) == 100
assert np.allclose(get_cumulative_rewards([0,0,1,0,0,1,0],gamma=0.9),[1.40049, 1.5561, 1.729, 0.81, 0.9, 1.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,-2,3,-4,0],gamma=0.5), [0.0625, 0.125, 0.25, -1.5, 1.0, -4.0, 0.0])
assert np.allclose(get_cumulative_rewards([0,0,1,2,3,4,0],gamma=0), [0, 0, 1, 2, 3, 4, 0])
print("looks good!")

looks good!


In [11]:
def train_step(_states,_actions,_rewards):
    """given full session, trains agent with policy gradient"""
    _cumulative_rewards = get_cumulative_rewards(_rewards)
    update.run({states:_states,actions:_actions,cumulative_rewards:_cumulative_rewards})

### Playing the game

In [12]:
def generate_session(t_max=1000):
    """play env with REINFORCE agent and train at the session end"""
    
    #arrays to record session
    states,actions,rewards = [],[],[]
    
    s = env.reset()
    
    for t in range(t_max):
        
        #action probabilities array aka pi(a|s)
        action_probas = get_action_proba(s)
        
        a = np.random.choice(n_actions, p=action_probas) #<pick random action using action_probas>
        
        new_s,r,done,info = env.step(a)
        
        #record session history to train later
        states.append(s)
        actions.append(a)
        rewards.append(r)
        
        s = new_s
        if done: break
            
    train_step(states,actions,rewards)
            
    return sum(rewards)
        

In [13]:
#s.close()
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())

for i in range(100):
    
    rewards = [generate_session() for _ in range(100)] #generate new sessions
    
    print ("mean reward:%.3f"%(np.mean(rewards)))

    if np.mean(rewards) > 300:
        print ("You Win!")
        break
        


mean reward:10.820
mean reward:9.760
mean reward:11.070
mean reward:13.100
mean reward:15.530
mean reward:21.950
mean reward:22.670
mean reward:23.710
mean reward:30.800
mean reward:34.950
mean reward:37.450
mean reward:48.100
mean reward:65.550
mean reward:124.380
mean reward:114.830
mean reward:131.610
mean reward:114.830
mean reward:159.500
mean reward:143.600
mean reward:112.370
mean reward:157.940
mean reward:167.750
mean reward:166.420
mean reward:161.210
mean reward:104.590
mean reward:131.680
mean reward:150.830
mean reward:132.730
mean reward:101.020
mean reward:127.180
mean reward:148.280
mean reward:256.470
mean reward:251.040
mean reward:244.050
mean reward:288.470
mean reward:272.330
mean reward:249.480
mean reward:258.030
mean reward:314.960
You Win!


### Results & video

In [14]:
#record sessions
import gym.wrappers
env = gym.wrappers.Monitor(gym.make("CartPole-v0"),directory="videos",force=True)
sessions = [generate_session() for _ in range(100)]
env.close()


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


NoSuchDisplayException: Cannot connect to "None"

In [14]:
#show video
from IPython.display import HTML
import os

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./videos/")))

HTML("""
<video width="640" height="480" controls>
  <source src="{}" type="video/mp4">
</video>
""".format("./videos/"+video_names[-1])) #this may or may not be _last_ video. Try other indices

In [ ]:
from submit import submit_cartpole
submit_cartpole(generate_session, 'pmatthews91@gmail.com', 'jqegGx19TK8x8BTi')

ConnectionError: HTTPSConnectionPool(host='www.coursera.org', port=443): Max retries exceeded with url: /api/onDemandProgrammingScriptSubmissions.v1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f4fe65a36d8>: Failed to establish a new connection: [Errno 110] Connection timed out',))

In [ ]:
# That's all, thank you for your attention!
# Not having enough? There's an actor-critic waiting for you in the honor section.
# But make sure you've seen the videos first.